In [2]:
import pandas as pd
import sqlite3
import numpy as np

In [ ]:
customers_df = pd.read_csv('customers.csv')
managers_df = pd.read_csv('managers.csv')
orders_df = pd.read_csv('SimulatedOrders.csv')
products_on_website_df = pd.read_csv('ProductsOnWebsite.csv')

db_conn = sqlite3.connect('BigBasket.db')

customers_df.to_sql('Customers', db_conn, if_exists='replace', index=False)
managers_df.to_sql('Managers', db_conn, if_exists='replace', index=False)
products_on_website_df.to_sql('ProductsOnWebsite', db_conn, if_exists='replace', index=False)
orders_df.to_sql('Orders', db_conn, if_exists='replace', index=False)

db_conn.close()


In [ ]:
products_df = pd.read_csv('ProductsOnWebsite.csv')
products_df = products_df.drop_duplicates(subset='ProductName', keep='first')

num_customers = 1000
orders_per_customer = 60
total_orders = num_customers * orders_per_customer

np.random.seed(42)

customer_ids = np.repeat(np.arange(1, num_customers + 1), orders_per_customer)
customer_ids = ['c' + str(i) for i in customer_ids] 
order_numbers = np.tile(np.arange(1, orders_per_customer + 1), num_customers)
order_ids = [f'{cid}-{num}' for cid, num in zip(customer_ids, order_numbers)]

all_orders = []

current_date = pd.Timestamp('today')
dates = pd.date_range(start=current_date - pd.DateOffset(years=1), end=current_date, periods=total_orders)
date_list = dates.tolist()
np.random.shuffle(date_list)

product_price_dict = products_df.set_index('ProductName')['DiscountPrice'].to_dict()

all_orders = []
for customer_id, order_id, date in zip(customer_ids, order_ids, date_list):
    num_items = np.random.randint(8, 21) 
    product_choices = np.random.choice(products_df['ProductName'], size=num_items)
    quantities = np.random.randint(1, 6, size=num_items)
    order_date = pd.Timestamp(date).strftime('%d/%m/%Y')

    prices = [product_price_dict[product] * quantity for product, quantity in zip(product_choices, quantities)]

    for product_name, quantity, price in zip(product_choices, quantities, prices):
        all_orders.append([customer_id, order_id, product_name, quantity, order_date, price])

orders_df = pd.DataFrame(all_orders, columns=['CustomerID', 'OrderID', 'ProductName', 'Quantity', 'OrderDate', 'Price'])

orders_csv_path = 'SimulatedOrders.csv'
orders_df.to_csv(orders_csv_path, index=False)

orders_csv_path, orders_df.head()
